In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [42]:
data = pd.read_csv('./data/real1/snv-parse-real1.txt', sep='\t', dtype={'Chr': str})
data['Chr'] = data['Chr'].astype('str') 

In [43]:
truth = pd.read_csv('./real1/real1_truth.bed', sep='\t', header=None)

In [29]:
data = pd.read_csv('./data/real1/snv-parse-real1.txt', sep='\t', dtype={'Chr': str})

truth = pd.read_csv('./data/real1/real1_truth.bed', sep='\t', header=None, names=['Chr', 'START_POS_REF', 'END_POS_REF'])

#'Chr' is treated as string in both datasets
data['Chr'] = data['Chr'].astype(str)
truth['Chr'] = truth['Chr'].astype(str)

# Create a set of true SNV positions from the BED file
true_snv_set = set(zip(truth['Chr'], truth['START_POS_REF'], truth['END_POS_REF']))

# Add a new column 'True_SNV' to indicate if the SNV is in ground truth
data['True_SNV'] = data.apply(lambda row: (row['Chr'], row['START_POS_REF'], row['END_POS_REF']) in true_snv_set, axis=1)

# Save the final dataset with the new column
data.to_csv('./data/real1/snv-parse-real1-labeled.txt', sep='\t', index=False)

In [66]:
data = pd.read_csv('./data/real1/snv-parse-real1-labeled.txt', sep='\t', dtype={'Chr': str})
data.head()

,Chr,START_POS_REF,END_POS_REF,REF,ALT,REF_MFVdVs,ALT_MFVdVs,Sample_Name,FILTER_Mutect2,FILTER_Freebayes,FILTER_Vardict,FILTER_Varscan,m2_MQ,f_MQMR,vs_SSC,vs_SPV,vd_SSF,vd_MSI,True_SNV
0,1,13110,13110,G,A,G/NA/G/G/,A/NA/A/A/,icgc_cll-T,True,False,False,False,41.91,NaN,2.0,0.522430,0.23427,2.0,False
1,1,15015,15015,G,C,G/NA/NA/G/,C/NA/NA/C/,icgc_cll-T,True,False,False,False,43.42,NaN,5.0,0.302390,NaN,NaN,False
2,1,16949,16949,A,C,NA/NA/NA/A/,NA/NA/NA/C/,icgc_cll-T,False,False,False,True,NaN,NaN,16.0,0.023282,NaN,NaN,False
3,1,40552,40552,T,C,NA/NA/NA/T/,NA/NA/NA/C/,icgc_cll-T,False,False,False,True,NaN,NaN,26.0,0.002231,NaN,NaN,False
4,1,46907,46907,T,C,NA/NA/NA/T/,NA/NA/NA/C/,icgc_cll-T,False,False,False,True,NaN,NaN,17.0,0.017670,NaN,NaN,False


In [67]:
# clean columns
# Drop non-numeric variables
non_numeric_cols = ['REF', 'ALT', 'REF_MFVdVs', 'ALT_MFVdVs', 'Sample_Name']
data = data.drop(columns=non_numeric_cols)
data.head()

,Chr,START_POS_REF,END_POS_REF,FILTER_Mutect2,FILTER_Freebayes,FILTER_Vardict,FILTER_Varscan,m2_MQ,f_MQMR,vs_SSC,vs_SPV,vd_SSF,vd_MSI,True_SNV
0,1,13110,13110,True,False,False,False,41.91,NaN,2.0,0.522430,0.23427,2.0,False
1,1,15015,15015,True,False,False,False,43.42,NaN,5.0,0.302390,NaN,NaN,False
2,1,16949,16949,False,False,False,True,NaN,NaN,16.0,0.023282,NaN,NaN,False
3,1,40552,40552,False,False,False,True,NaN,NaN,26.0,0.002231,NaN,NaN,False
4,1,46907,46907,False,False,False,True,NaN,NaN,17.0,0.017670,NaN,NaN,False


In [70]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
X

array([['1', 13110, 13110, ..., 0.52243, 0.23427, 2.0],
       ['1', 15015, 15015, ..., 0.30239, nan, nan],
       ['1', 16949, 16949, ..., 0.023282, nan, nan],
       ...,
       ['Y', 59030255, 59030255, ..., nan, nan, nan],
       ['Y', 59031439, 59031439, ..., 0.52571, 0.54983, 1.0],
       ['Y', 59031799, 59031799, ..., nan, nan, nan]],
      shape=(49320, 13), dtype=object)

In [77]:
# Clean the data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
data = pd.read_csv('./data/real1/snv-parse-real1-labeled.txt', sep='\t', dtype={'Chr': str})
non_numeric_cols = ['REF', 'ALT', 'REF_MFVdVs', 'ALT_MFVdVs', 'Sample_Name']
data = data.drop(columns=non_numeric_cols)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# One-hot encode 'Chr'
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = ct.fit_transform(X).toarray()  # Convert to numpy array
print(X)  # Print the first row

[[1.       0.       0.       ... 0.52243  0.23427  2.      ]
 [1.       0.       0.       ... 0.30239       nan      nan]
 [1.       0.       0.       ... 0.023282      nan      nan]
 ...
 [0.       0.       0.       ...      nan      nan      nan]
 [0.       0.       0.       ... 0.52571  0.54983  1.      ]
 [0.       0.       0.       ...      nan      nan      nan]]


In [78]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [79]:
from sklearn.svm import SVC

classifier = SVC(kernel='linear', random_state=0) # classic model with the linear kernel
model = classifier.fit(X_train, y_train)
print(model)

ValueError: Input X contains NaN.
SVC does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
confusion = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
f1